<a href="https://colab.research.google.com/github/langtheworld/Mukapon-Py1/blob/main/6670314321_XGBoosting_MidTerm_bonus_final_send.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gensim

In [ ]:
import re
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#1. Load data

In [ ]:
# Replace 'your_file.json' with the actual file name
trainfile_name = 'train_student_bonus.json'

# Open and read the JSON file
with open(trainfile_name, 'r') as json_file:
    traindata = json.load(json_file)

In [ ]:
pd.DataFrame(traindata)

,001,002,003,004,005,006,007,008,009,010,...,445,446,447,448,449,450,451,452,453,454
Title,Activated carbon derived from bacterial cellul...,The algorithm of static hand gesture recogniti...,Alternative Redundant Residue Number System Co...,Comparative study of wax inhibitor performance...,Undrained lower bound solutions for end bearin...,Words Diffusion an Analysis of across Facebook...,Transformation of time Petri net into Promela,Annual Degradation Rate Analysis of Mono-Si Ph...,Development of Low-Cost in-the-Ear EEG Prototype,Model-based analysis of an integrated zinc-air...,...,Effect of rhenium and cobalt additions on the ...,Wheeling charge calculation with consideration...,Scaling laws for static displacement of linear...,Effect of TMB/P123 ratios on physicochemical p...,Synthetic CaO-based sorbent for high-temperatu...,A portable USB-controlled potentiostat for pap...,Literature reviews on applying artificial inte...,A multi-parameterized water quality prediction...,Semantic Segmentation on Medium-Resolution Sat...,Reducing the defects of a-pillar stamping part...
Abstract,© 2019 Elsevier B.V.Activated carbon derived f...,© Springer International Publishing AG 2018.Te...,© 2018 IEEE.Residue number system (RNS) is a n...,© Published under licence by IOP Publishing Lt...,"© 2019 John Wiley & Sons, Ltd.The undrained be...",© 2018 IEEE.Facebook Pages in Thailand have be...,© 2017 IEEE.This paper proposes a method of tr...,© 2013 IEEE.The annual degradation rate (DR) o...,© 2018 IEEE.This study focused on building a l...,"© 2019 Lao-atiman, Bumroongsil, Arpornwichanop...",...,© Carl Hanser Verlag GmbH & Co. KG.The effect ...,"© 2019 IEEE.In Thailand, Small Power Producers...",© 2018 Elsevier LtdExperimental studies on the...,© 2021 Elsevier LtdThe preparation of mesocell...,© 2018 Hydrogen Energy Publications LLCCalcium...,© 2018 IEEEThis paper presents a portable and ...,Copyright © 2019 for this paper by its authors...,© 2019 The authors and IOS Press. All rights r...,© 2018 IEEE.Semantic Segmentation is a fundame...,© 2019 IEEE.This research aims to reduce defec...
Classes,[CHE],[CPE],[EE],"[CHE, ME, AUTO]","[CE, IE]",[CPE],"[CPE, IE]","[EE, CHE, ME]","[CE, EE, CPE, IE]","[EE, CHE, ME, IE]",...,"[CHE, ME, IE, AUTO]","[EE, AUTO]","[CE, IE]","[EE, CHE, IE]","[CHE, ME]","[EE, CPE, CHE]","[CPE, IE]","[CE, EE, CHE]","[EE, CPE, IE, AUTO]","[ME, IE]"


In [ ]:
# format to data table
df = pd.DataFrame(traindata).T.reset_index()

df

,index,Title,Abstract,Classes
0,001,Activated carbon derived from bacterial cellul...,© 2019 Elsevier B.V.Activated carbon derived f...,[CHE]
1,002,The algorithm of static hand gesture recogniti...,© Springer International Publishing AG 2018.Te...,[CPE]
2,003,Alternative Redundant Residue Number System Co...,© 2018 IEEE.Residue number system (RNS) is a n...,[EE]
3,004,Comparative study of wax inhibitor performance...,© Published under licence by IOP Publishing Lt...,"[CHE, ME, AUTO]"
4,005,Undrained lower bound solutions for end bearin...,"© 2019 John Wiley & Sons, Ltd.The undrained be...","[CE, IE]"
...,...,...,...,...
449,450,A portable USB-controlled potentiostat for pap...,© 2018 IEEEThis paper presents a portable and ...,"[EE, CPE, CHE]"
450,451,Literature reviews on applying artificial inte...,Copyright © 2019 for this paper by its authors...,"[CPE, IE]"
451,452,A multi-parameterized water quality prediction...,© 2019 The authors and IOS Press. All rights r...,"[CE, EE, CHE]"
452,453,Semantic Segmentation on Medium-Resolution Sat...,© 2018 IEEE.Semantic Segmentation is a fundame...,"[EE, CPE, IE, AUTO]"


#2. Data preprocessing

##2.1 Cleansing data text แสดงผลเฉพาะตัวอักษรออกมา

In [ ]:
import nltk
# Uncomment to download "stopwords"
nltk.download("stopwords")
from nltk.corpus import stopwords

def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Change "'t" to "not"
    - Remove "@name"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    """
    s = s.lower()
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    # Remove Number
    s = re.sub('[0-9]+', '', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\!\?\(\)\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('english')
                  or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()

    return s

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# add new column which is Title+Abstract
df['TitleAbstract'] =  df['Title'] + df['Abstract']

In [ ]:
df['Title_clean'] = df['Title'].apply(text_preprocessing)
df['Abstract_clean'] = df['Abstract'].apply(text_preprocessing)

In [ ]:
# Cleansing data in TitleAbstract
df['TitleAbstract_clean'] = df['TitleAbstract'].apply(text_preprocessing)

##2.2 Transform Y data

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
# Create a MultiLabelBinarizer instance
mlb = MultiLabelBinarizer()

In [ ]:
# Fit and transform the labels into a binary matrix
one_hot_labels = mlb.fit_transform(df['Classes'])

# Create a new DataFrame with one-hot encoded labels
one_hot_df = pd.DataFrame(one_hot_labels, columns=mlb.classes_)

# Connect Column one hot label together
data_clean = pd.concat([df, one_hot_df], axis=1)

In [ ]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   index                454 non-null    object
 1   Title                454 non-null    object
 2   Abstract             454 non-null    object
 3   Classes              454 non-null    object
 4   TitleAbstract        454 non-null    object
 5   Title_clean          454 non-null    object
 6   Abstract_clean       454 non-null    object
 7   TitleAbstract_clean  454 non-null    object
 8   AUTO                 454 non-null    int64 
 9   CE                   454 non-null    int64 
 10  CHE                  454 non-null    int64 
 11  CPE                  454 non-null    int64 
 12  EE                   454 non-null    int64 
 13  IE                   454 non-null    int64 
 14  ME                   454 non-null    int64 
dtypes: int64(7), object(8)
memory usage: 53.3+ KB


#2.3 TF-IDF:Word to Vector

In [ ]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocess text
data_preprocessed = np.array(data_clean['Title_clean'])

# Calculate TF-IDF
tf_idf = TfidfVectorizer(ngram_range=(1, 3),
                         binary=True,
                         smooth_idf=False)
data_train_tfidf = tf_idf.fit_transform(data_preprocessed)

CPU times: user 44.8 ms, sys: 4.98 ms, total: 49.8 ms
Wall time: 56.1 ms


#3. Model

In [ ]:
import xgboost as xgb

In [ ]:
# Create an XGBoost multi-label classifier
clf = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False  # XGBoost 1.0+ doesn't require label encoding
)

In [ ]:
# Fit the model to the training data
clf.fit(data_train_tfidf, data_clean.loc[:,'AUTO':])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

#Test data

In [ ]:
# Replace 'your_file.json' with the actual file name
testFIle_name = 'test_student_bonus.json'

# Open and read the JSON file
with open(testFIle_name, 'r') as json_file:
    testdata = json.load(json_file)

In [ ]:
# format to data table
test_data = pd.DataFrame(testdata).T
test_data = test_data.reset_index()

test_data

,index,Title,Abstract,Classes
0,001eval,Comparative Electrical Energy Yield Performanc...,© 2013 IEEE.Long-term energy evaluation of PV ...,"[CHE, IE]"
1,002eval,Effects of graphene nanoplatelets on bio-based...,© The Author(s) 2021.Novel near-infrared (NIR)...,"[CHE, IE]"
2,003eval,Anti-inflammatory action of two novel peptides...,© The Royal Society of Chemistry 2020.Peanut w...,"[CE, CHE]"
3,004eval,Efficient all-and-one support vector machines ...,© 2018 IEEE.We introduce a new strategy to est...,"[CPE, IE]"
4,005eval,Driver identification using histogram and neur...,© 2017 IEEE.Sensor technology has continuously...,"[CPE, IE]"
...,...,...,...,...
146,147eval,Utilization of Sewage Sludge from Beverage Ind...,© Published under licence by IOP Publishing Lt...,"[CE, CHE]"
147,148eval,Development of a Gateway for OpenADR-ECHONET L...,"© 2018 IEEE.In this paper, we develop an ECHON...","[EE, AUTO]"
148,149eval,Effect of solution treatment and precipitation...,© 2017 Elsevier Ltd. All rights reserved.The a...,"[CHE, ME, AUTO]"
149,150eval,An effect-analysis method for species-dependen...,"© The Authors, published by EDP Sciences, 2019...","[CE, CHE, AUTO]"


In [ ]:
7# Preprocessing Test data
test_data['TitleAbstract_clean'] = test_data['Title'] + test_data['Abstract']
test_data['Title_clean'] = test_data['Title'].apply(text_preprocessing)
test_data['Title_clean'] = np.array(test_data['Title'].apply(text_preprocessing))

#Array abstract
test_data['Abstract_clean'] = test_data['Abstract'].apply(text_preprocessing)
test_data['Abstract_clean'] = np.array(test_data['Abstract'].apply(text_preprocessing))
test_data['TitleAbstract_clean'] = np.array(test_data['TitleAbstract_clean'].apply(text_preprocessing))

data_test = tf_idf.transform(test_data['Abstract_clean'])

In [ ]:
data_test

<151x9649 sparse matrix of type '<class 'numpy.float64'>'
	with 5817 stored elements in Compressed Sparse Row format>

In [ ]:
mlb2 = MultiLabelBinarizer()
# Fit and transform the labels into a binary matrix
one_hot_labels_test = mlb2.fit_transform(test_data['Classes'])

# Create a new DataFrame with one-hot encoded labels
one_hot_df_test = pd.DataFrame(one_hot_labels_test, columns=mlb2.classes_)

# Connect Column one hot label together
data_clean_test = pd.concat([test_data, one_hot_df_test], axis=1)

In [ ]:
data_clean_test

,index,Title,Abstract,Classes,TitleAbstract_clean,Title_clean,Abstract_clean,AUTO,CE,CHE,CPE,EE,IE,ME
0,001eval,Comparative Electrical Energy Yield Performanc...,© 2013 IEEE.Long-term energy evaluation of PV ...,"[CHE, IE]",comparative electrical energy yield performanc...,comparative electrical energy yield performanc...,ieee long term energy evaluation pv systems us...,0,0,1,0,0,1,0
1,002eval,Effects of graphene nanoplatelets on bio-based...,© The Author(s) 2021.Novel near-infrared (NIR)...,"[CHE, IE]",effects graphene nanoplatelets bio based shape...,effects graphene nanoplatelets bio based shape...,author novel near infrared nir light induced b...,0,0,1,0,0,1,0
2,003eval,Anti-inflammatory action of two novel peptides...,© The Royal Society of Chemistry 2020.Peanut w...,"[CE, CHE]",anti inflammatory action two novel peptides de...,anti inflammatory action two novel peptides de...,royal society chemistry peanut worm sipunculus...,0,1,1,0,0,0,0
3,004eval,Efficient all-and-one support vector machines ...,© 2018 IEEE.We introduce a new strategy to est...,"[CPE, IE]",efficient one support vector machines based on...,efficient one support vector machines based on...,ieee introduce new strategy estimate data inse...,0,0,0,1,0,1,0
4,005eval,Driver identification using histogram and neur...,© 2017 IEEE.Sensor technology has continuously...,"[CPE, IE]",driver identification using histogram neural n...,driver identification using histogram neural n...,ieee sensor technology continuously improved t...,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,147eval,Utilization of Sewage Sludge from Beverage Ind...,© Published under licence by IOP Publishing Lt...,"[CE, CHE]",utilization sewage sludge beverage industry dy...,utilization sewage sludge beverage industry dy...,published licence iop publishing ltd sewage sl...,0,1,1,0,0,0,0
147,148eval,Development of a Gateway for OpenADR-ECHONET L...,"© 2018 IEEE.In this paper, we develop an ECHON...","[EE, AUTO]",development gateway openadr echonet lite proto...,development gateway openadr echonet lite proto...,ieee paper develop echonet lite gateway raspbe...,1,0,0,0,1,0,0
148,149eval,Effect of solution treatment and precipitation...,© 2017 Elsevier Ltd. All rights reserved.The a...,"[CHE, ME, AUTO]",effect solution treatment precipitation aging ...,effect solution treatment precipitation aging ...,elsevier ltd rights reserved aim project study...,1,0,1,0,0,0,1
149,150eval,An effect-analysis method for species-dependen...,"© The Authors, published by EDP Sciences, 2019...","[CE, CHE, AUTO]",effect analysis method species dependent coral...,effect analysis method species dependent coral...,authors published edp sciences research aimed ...,1,1,1,0,0,0,0


In [ ]:
data_clean_test

,index,Title,Abstract,Classes,TitleAbstract_clean,Title_clean,Abstract_clean,AUTO,CE,CHE,CPE,EE,IE,ME
0,001eval,Comparative Electrical Energy Yield Performanc...,© 2013 IEEE.Long-term energy evaluation of PV ...,"[CHE, IE]",comparative electrical energy yield performanc...,comparative electrical energy yield performanc...,ieee long term energy evaluation pv systems us...,0,0,1,0,0,1,0
1,002eval,Effects of graphene nanoplatelets on bio-based...,© The Author(s) 2021.Novel near-infrared (NIR)...,"[CHE, IE]",effects graphene nanoplatelets bio based shape...,effects graphene nanoplatelets bio based shape...,author novel near infrared nir light induced b...,0,0,1,0,0,1,0
2,003eval,Anti-inflammatory action of two novel peptides...,© The Royal Society of Chemistry 2020.Peanut w...,"[CE, CHE]",anti inflammatory action two novel peptides de...,anti inflammatory action two novel peptides de...,royal society chemistry peanut worm sipunculus...,0,1,1,0,0,0,0
3,004eval,Efficient all-and-one support vector machines ...,© 2018 IEEE.We introduce a new strategy to est...,"[CPE, IE]",efficient one support vector machines based on...,efficient one support vector machines based on...,ieee introduce new strategy estimate data inse...,0,0,0,1,0,1,0
4,005eval,Driver identification using histogram and neur...,© 2017 IEEE.Sensor technology has continuously...,"[CPE, IE]",driver identification using histogram neural n...,driver identification using histogram neural n...,ieee sensor technology continuously improved t...,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,147eval,Utilization of Sewage Sludge from Beverage Ind...,© Published under licence by IOP Publishing Lt...,"[CE, CHE]",utilization sewage sludge beverage industry dy...,utilization sewage sludge beverage industry dy...,published licence iop publishing ltd sewage sl...,0,1,1,0,0,0,0
147,148eval,Development of a Gateway for OpenADR-ECHONET L...,"© 2018 IEEE.In this paper, we develop an ECHON...","[EE, AUTO]",development gateway openadr echonet lite proto...,development gateway openadr echonet lite proto...,ieee paper develop echonet lite gateway raspbe...,1,0,0,0,1,0,0
148,149eval,Effect of solution treatment and precipitation...,© 2017 Elsevier Ltd. All rights reserved.The a...,"[CHE, ME, AUTO]",effect solution treatment precipitation aging ...,effect solution treatment precipitation aging ...,elsevier ltd rights reserved aim project study...,1,0,1,0,0,0,1
149,150eval,An effect-analysis method for species-dependen...,"© The Authors, published by EDP Sciences, 2019...","[CE, CHE, AUTO]",effect analysis method species dependent coral...,effect analysis method species dependent coral...,authors published edp sciences research aimed ...,1,1,1,0,0,0,0


In [ ]:
# Make predictions on the test data
y_pred = clf.predict(data_test)

In [ ]:
data_pred = pd.DataFrame(y_pred, index=test_data.index, columns=['AUTO', 'CE' , 'CHE' , 'CPE', 'EE','IE','ME'])

data_pred

,AUTO,CE,CHE,CPE,EE,IE,ME
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,1.0,1.0,1.0,0.0
4,0.0,0.0,0.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...
146,0.0,1.0,1.0,1.0,1.0,1.0,0.0
147,1.0,0.0,0.0,1.0,0.0,1.0,0.0
148,0.0,0.0,1.0,0.0,0.0,1.0,0.0
149,0.0,0.0,0.0,1.0,1.0,1.0,0.0


In [ ]:
desired_columns = ['AUTO', 'CE' , 'CHE' , 'CPE', 'EE','IE','ME']
data_pred = data_pred[desired_columns]
data_pred = data_pred.astype(int)
data_pred.insert(0, 'id', test_data.index)
data_pred

,id,AUTO,CE,CHE,CPE,EE,IE,ME
0,0,0,1,0,1,0,1,0
1,1,0,0,1,0,0,1,0
2,2,0,0,0,0,0,1,0
3,3,0,0,1,1,1,1,0
4,4,0,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...
146,146,0,1,1,1,1,1,0
147,147,1,0,0,1,0,1,0
148,148,0,0,1,0,0,1,0
149,149,0,0,0,1,1,1,0


In [ ]:
data_y_test = data_clean_test[desired_columns]
data_y_test

,AUTO,CE,CHE,CPE,EE,IE,ME
0,0,0,1,0,0,1,0
1,0,0,1,0,0,1,0
2,0,1,1,0,0,0,0
3,0,0,0,1,0,1,0
4,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...
146,0,1,1,0,0,0,0
147,1,0,0,0,1,0,0
148,1,0,1,0,0,0,1
149,1,1,1,0,0,0,0


#save To csv

In [ ]:
data_pred.to_csv('data_pred_exam_bonus.csv', index=False)

In [ ]:
data_clean.to_csv('data_clean_input_bonus.csv')

In [ ]:
clf.save_model('xgboost_model_bonus.model')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [14:04:50] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,ConfusionMatrixDisplay
from sklearn.metrics import multilabel_confusion_matrix
accuracy = accuracy_score(data_y_test, y_pred)
confusion = multilabel_confusion_matrix(data_y_test, y_pred)
classification_rep = classification_report(data_y_test, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(classification_rep)

              precision    recall  f1-score   support

           0       0.20      0.32      0.25        25
           1       0.56      0.50      0.53        56
           2       0.68      0.78      0.72        67
           3       0.41      0.32      0.36        37
           4       0.56      0.66      0.60        67
           5       0.61      0.83      0.70        93
           6       0.43      0.51      0.47        41

   micro avg       0.54      0.63      0.58       386
   macro avg       0.49      0.56      0.52       386
weighted avg       0.54      0.63      0.58       386
 samples avg       0.53      0.63      0.54       386



In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=confusion,display_labels=clf.classes_)